In [11]:
from dotenv import load_dotenv
load_dotenv()


False

In [12]:
%pip install psycopg[binary]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import os
import psycopg

# Load DATABASE_URL from environment variable
os.chdir("..")
print("Current working directory:", os.getcwd())
print("Directory contents:", os.listdir())

DATABASE_URL = os.getenv("DATABASE_URL")

if not DATABASE_URL:
    raise RuntimeError("DATABASE_URL environment variable not set. Please set it to your Neon database connection string.")

# Connect to the Neon Postgres database
try:
    conn = psycopg.connect(DATABASE_URL)
    print("✅ Successfully connected to Neon database!")
except Exception as e:
    print("❌ Failed to connect to Neon database:", e)
    conn = None


Current working directory: c:\Users\Dries\Desktop
Directory contents: ['Among Us.url', 'Atradius.csv', 'Atradius_questions.xlsx', 'bestemmingx', 'blog', 'Call of Duty® Modern Warfare® II.url', 'CapCut.lnk', 'coding test', 'Cursor.lnk', 'desktop.ini', 'Discord.lnk', 'DNA Notifyer', 'Dries - Chrome.lnk', 'env.txt', 'Fall Guys.url', 'Fierce', 'FifaScraper', 'FifaScraper.zip', 'League of Legends PBE.lnk', 'LM Studio.lnk', 'MongoDBCompass.lnk', 'Nipro S2S', 'OptiFine_1.16.5_HD_U_G7.jar', 'python_test', 'Razer Cortex.lnk', 'RedditScraper', 'Riot Client.lnk', 'roulette', 'SaaS Ideator', 'Stremio.lnk', 'Thesis Arthur', 'Thesis Arthur.docx', 'transformers', 'van-breda-data-scraping', 'WantedLaw', 'Webscraper2.0']
✅ Successfully connected to Neon database!


In [14]:
if conn:
    with conn.cursor() as cur:
        # Get column names
        cur.execute("SELECT column_name FROM information_schema.columns WHERE table_name = 'silver_classes' AND table_schema = 'public';")
        columns = [row[0] for row in cur.fetchall()]

        # For each column, count non-null values grouped by 'source'
        print("Non-null value counts for each column, grouped by 'source' (excluding sources containing '%test%'):")
        for col in columns:
            if col == 'source':
                continue  # skip grouping column itself
            query = f"""
                SELECT source, COUNT({col}) AS non_null_count
                FROM public.silver_classes
                WHERE {col} IS NOT NULL
                  AND source NOT ILIKE '%test%'
                GROUP BY source
                ORDER BY source;
            """
            cur.execute(query)
            results = cur.fetchall()
            print(f"\nColumn: {col}")
            if results:
                for source, count in results:
                    print(f"  Source: {source} | Non-null count: {count}")
            else:
                print("  No non-null values found for this column.")


Non-null value counts for each column, grouped by 'source' (excluding sources containing '%test%'):

Column: class_id
  Source: coolcharm | Non-null count: 1425
  Source: koepel | Non-null count: 401
  Source: rite | Non-null count: 4074
  Source: rowreformer | Non-null count: 4727

Column: class_name
  Source: coolcharm | Non-null count: 1425
  Source: koepel | Non-null count: 401
  Source: rite | Non-null count: 4074
  Source: rowreformer | Non-null count: 4727

Column: instructor
  Source: coolcharm | Non-null count: 46
  Source: koepel | Non-null count: 387
  Source: rite | Non-null count: 4074
  Source: rowreformer | Non-null count: 4674

Column: location
  Source: coolcharm | Non-null count: 1425
  Source: koepel | Non-null count: 401
  Source: rite | Non-null count: 4074
  Source: rowreformer | Non-null count: 4727

Column: start_ts
  Source: coolcharm | Non-null count: 1422
  Source: koepel | Non-null count: 400
  Source: rite | Non-null count: 3892
  Source: rowreformer | Non-

In [15]:
if conn:
    with conn.cursor() as cur:
        # Get all unique sources (excluding test sources)
        cur.execute("""
            SELECT DISTINCT source
            FROM public.silver_classes
            WHERE source NOT ILIKE '%test%'
            ORDER BY source;
        """)
        sources = [row[0] for row in cur.fetchall()]
        print("\nShowing 10 random rows for each source (excluding sources containing '%test%'):\n")
        for source in sources:
            print(f"Source: {source}")
            cur.execute(f"""
                SELECT *
                FROM public.silver_classes
                WHERE source = %s
                ORDER BY random()
                LIMIT 10;
            """, (source,))
            rows = cur.fetchall()
            if rows:
                # Print column headers
                colnames = [desc[0] for desc in cur.description]
                print(" | ".join(colnames))
                for row in rows:
                    print(" | ".join(str(val) if val is not None else "" for val in row))
            else:
                print("  No rows found for this source.")
            print("-" * 60)



Showing 10 random rows for each source (excluding sources containing '%test%'):

Source: coolcharm
class_id | source | class_name | instructor | location | start_ts | end_ts | capacity | spots_available | status | url | first_seen_at | last_updated_at | last_scraped_at | is_cancelled | is_past | source_run_id | source_snapshot_id | raw_data
coolcharm:505303974878 | coolcharm | Reformer (All Levels) |  | Antwerp | 2025-06-04 19:30:00+00:00 | 2025-06-04 20:20:00+00:00 |  |  | Book |  | 2025-09-01 11:05:05.256785+00:00 | 2025-09-01 11:05:05.256785+00:00 | 2025-05-23 16:03:34+00:00 | False | True | 11368a83-e282-4278-98d4-ef06fa4d43a1 | 119818 | {'date': 'WEDNESDAY 4 JUNE', 'time': '19:30 - 20:20', 'type': 'Reformer (All Levels)', 'status': 'Book', 'location': 'Antwerp'}
coolcharm:753144622347 | coolcharm | Reformer (All Levels) |  | Antwerp | 2025-06-24 17:30:00+00:00 | 2025-06-24 18:20:00+00:00 |  |  | Book |  | 2025-09-01 11:08:08.328618+00:00 | 2025-09-01 11:08:08.328618+00:00 | 2025-